This notebook is was used to figure out how to solve the portfolio optimization problem using a LP/MIP/QP solver.  This notebook contains a prototype that attempts to use [MIOSQP](https://github.com/osqp/miosqp) to solve the problem.

The problem requires support for:

* Quadratic programming (QP) - supports minimizing a quadratic objective function (i.e., sum of the squared difference)
* Mixed-integer programming - supports integer variables (i.e., number of funds)
* Linear constraints - support constraining overall allocation to 100% and the allocation to any specific asset class or fund to be less than 100%
Because of these requirements, the solver needs to support mixed-integer quadratic programming (MIQP).

I was unable to get MIOSQP to work.  Documentation is nearly non-existent and the code does not seem to be well maintained.  The example code uses the non-existent Scipy method (`randn`) and doesn't appear to work.  There seems to be incompatibilities between the settings reqiuired by MIOSQP and setting allowed by OSQP - causing a circular loop of errors (one saying a particular setting is required and the other saying that setting is not allowed). :-(

In [ ]:
# --
# Load data from csv
# --
import pandas as pd

In [ ]:
file_path = "../data/exposure_matrix.csv"

# Read only the header row
headers = pd.read_csv(file_path, nrows=0).columns.tolist()

# Define the default dtype for all columns except 'Ticker'
dtype_dict = {col: float for col in headers if col != 'Ticker'}

# Read the full file with the dynamically created dtype and converter
data = pd.read_csv(
    file_path,
    dtype=dtype_dict,  # Set all columns to float except Ticker
    converters={'Ticker': lambda x: x.strip()}  # Strip whitespace from Ticker column
)
data.set_index('Ticker', inplace=True)
data.loc['BNDX']
data.loc['BNDX', 'Intl Bonds']
data.loc[:, 'Intl Bonds']
data

In [ ]:
# extract_data(data):

# Extract fund_matrix (all rows except the footer)
fund_matrix = data.iloc[:-1]
fund_matrix.loc['BNDX']
fund_matrix.loc[:,'Cash']
fund_matrix.loc['BNDX','Cash']
fund_matrix

In [ ]:
# Extract asset_class_targets (footer row)
asset_class_targets = data.loc['Targets']
asset_class_targets.loc['Emerging']
asset_class_targets

In [ ]:
# Extract fund tickers (first column)
funds = fund_matrix.index
funds

In [ ]:
# Extract asset classes (header row, excluding the first column)
asset_classes = data.columns
asset_classes

**Problem**:

**Minimize the following**:
* sum of the squared difference between final portfolio asset class allocations and target asset class allocations
* the number of funds included in the portfolio (# of funds with non-zero allocations)

**Subject to**:
* sum of the final portfolio asset class allocations equals 1
* sum of the final portfolio fund allocations equals 1# - sum of the portfolio asset allocations equals 1
* portfolio allocation for each asset class is less than 1
* portfolio allocation for each fund is less than 1
* number of funds included in the portfolio is less than max_funds

In [1]:
#import numpy as np
from scipy import sparse

# simple example
# 3 funds, 4 asset classes
# rows -> funds, columns -> asset_classes
# rows must sum to 100%

exposure_matrix = sparse.csc_matrix([[0.25, 0.3, 0.4, 0.05],
                                     [0.15, 0.2, 0.5, 0.15],
                                     [0.6,  0.0, 0.3, 0.1]])
print(exposure_matrix)

# desired portfolio asset class allocation
target_exposures = sparse.csc_matrix([0.1, 0.2, 0.3, 0.4])
print(target_exposures)

# actual fund allocations
fund_allocations = sparse.csc_matrix([0.2, 0.3, 0.5])
print(fund_allocations)

# actual portfolio asset class allocations



<Compressed Sparse Column sparse matrix of dtype 'float64'
	with 11 stored elements and shape (3, 4)>
  Coords	Values
  (0, 0)	0.25
  (1, 0)	0.15
  (2, 0)	0.6
  (0, 1)	0.3
  (1, 1)	0.2
  (0, 2)	0.4
  (1, 2)	0.5
  (2, 2)	0.3
  (0, 3)	0.05
  (1, 3)	0.15
  (2, 3)	0.1
<Compressed Sparse Column sparse matrix of dtype 'float64'
	with 4 stored elements and shape (1, 4)>
  Coords	Values
  (0, 0)	0.1
  (0, 1)	0.2
  (0, 2)	0.3
  (0, 3)	0.4
<Compressed Sparse Column sparse matrix of dtype 'float64'
	with 3 stored elements and shape (1, 3)>
  Coords	Values
  (0, 0)	0.2
  (0, 1)	0.3
  (0, 2)	0.5


In [ ]:
# brute force calculation of sum of the difference squared
# iterate over asset classes
num_funds = exposure_matrix.shape[0]
num_classes = exposure_matrix.shape[1]

portfolio_exposures = np.zeros(num_classes)
diff_squared = np.zeros(num_classes)

# convert sparse arrays to dense arrays
e_matrix = exposure_matrix.toarray()
f_allocations = fund_allocations.toarray().flatten()
t_exposures = target_exposures.toarray().flatten()

print("e_matrix:")
print(e_matrix)
print("f_allocations:")
print(f_allocations)

for i in range(num_classes):
    for j in range(num_funds):
        portfolio_exposures[i] = portfolio_exposures[i] + (e_matrix[j, i] * f_allocations[j])
    diff_squared[i] = (portfolio_exposures[i] - t_exposures[i]) ** 2

print("portfolio_exposures:")
print(portfolio_exposures)

print("fund_allocations @ exposure_matrix")
pexposures = fund_allocations @ exposure_matrix
print(pexposures)

print("sum elements of portfolio exposures: (should equal 1)")
print(np.sum(pexposures))

print("diff_squared:")
print(diff_squared)

print("sum of diff squared:")
print(np.sum(diff_squared))


In [41]:
print(target_exposures.T.toarray())
print(exposure_matrix.toarray())
#target_exposures.T.toarray() @ exposure_matrix.toarray()
print(exposure_matrix.toarray() @ target_exposures.T.toarray())
print(target_exposures.toarray() @ exposure_matrix.T.toarray())

[[0.1]
 [0.2]
 [0.3]
 [0.4]]
[[0.25 0.3  0.4  0.05]
 [0.15 0.2  0.5  0.15]
 [0.6  0.   0.3  0.1 ]]
[[0.225]
 [0.265]
 [0.19 ]]
[[0.225 0.265 0.19 ]]


In [44]:
import numpy as np
import miosqp
def setup_and_solve(exposure_matrix, target_allocations):
    """
    Set up and solve the portfolio optimization problem using MIOSQP.
    - Minimize the sum of squared differences between fund allocations and target allocations.
    """
    n_funds = exposure_matrix.shape[0]
    #exposure_matrix = fund_matrix.values.T  # Transpose to match dimensions

    # Objective: sum of the squared difference between portfolio asset class
    # allocation and the target asset class allocation
    #
    # Let
    #   n = number of funds
    #   m = number of asset classes
    #   E = n x m matrix of asset class allocations for each fund (exposure_matrix)
    #   b = length m vector of target asset class allocations (target_allocations)
    #   x = length n vector of fund allocations (variables to be optimized)
    #
    # Given these definitions:
    #   Ex = vector of length m with portfolio's asset class allocation (given values for x)
    #   (Ex - b)^2 = the objective function
    #
    # The objective function becomes:
    #   (Ex - b)^2 = (Ex - b)^T(Ex - b) = x^T(E^TE)x - 2(b^TE)x + b^Tb
    #
    # The b^Tb term is a constant factort that does not impact the optimal values.
    #
    # The solver will optimize the equation: (1/2)x^TQx + c^Tx
    #
    # Let:
    #   Q = 2(E^TE) - the leading "2" cancels the (1/2) in the optimzation equation
    #   c = -2(b^TE)
    #
    Q = 2 * (exposure_matrix.T @ exposure_matrix)
    print(target_allocations)
    print(exposure_matrix)
    # c = -2 * (target_allocations.toarray() @ exposure_matrix.T.toarray())
    c = -2 * (exposure_matrix.toarray() @ target_allocations.T.toarray())
    print(c.shape)
    print(f'c: {c}')
    print(f'length of c: {len(c)}')
    print(f'n_funds: {n_funds}')

    # Constraints: constraints are defined using the following equation:
    #
    #   l <= Ax <= u
    #
    # To constrain the fund allocations to sum to 1:
    #
    # (1) Define A to be a length n vector of 1's.  Then:
    #
    #   Ax = A_1*x_1 + A_2*x_2 + ... + A_n*x_n = sum of fund allocations
    #
    # (2) Set l and u both equal to 1
    #
    # Then:
    #
    #   1 <= Ax <= 1
    A = np.ones((1, n_funds)) # length n vector of 1's
    print(f'A: {A}')
    l = np.array([1.0])  # lower bound
    u = np.array([1.0])  # upper bound

    # Define problem settings
    # Taken from: https://github.com/osqp/miosqp/blob/master/examples/random_miqp/run_example.py#L98
    miosqp_settings = {
        # integer feasibility tolerance
        'eps_int_feas': 1e-03,
        # maximum number of iterations
        'max_iter_bb': 1000,
        # tree exploration rule
        #   [0] depth first
        #   [1] two-phase: depth first until first incumbent and then  best bound
        'tree_explor_rule': 1,
        # branching rule
        #   [0] max fractional part
        'branching_rule': 0,
        'verbose': True,
        'print_interval': 1}

    osqp_settings = {
        'eps_abs': 1e-03,
        'eps_rel': 1e-03,
        'eps_prim_inf': 1e-04,
        'verbose': True}

    # Initialize and solve with MIOSQP
    miosqp_solver = miosqp.MIOSQP()
    miosqp_solver.setup(Q, c, A, l, u,
                        i_idx = [], i_l = [], i_u = [], 
                        settings = osqp_settings,
                        qp_settings = miosqp_settings)  # No integer constraints for now
    result = miosqp_solver.solve()

    return result

In [45]:
setup_and_solve(exposure_matrix, target_exposures)

<Compressed Sparse Column sparse matrix of dtype 'float64'
	with 4 stored elements and shape (1, 4)>
  Coords	Values
  (0, 0)	0.1
  (0, 1)	0.2
  (0, 2)	0.3
  (0, 3)	0.4
<Compressed Sparse Column sparse matrix of dtype 'float64'
	with 11 stored elements and shape (3, 4)>
  Coords	Values
  (0, 0)	0.25
  (1, 0)	0.15
  (2, 0)	0.6
  (0, 1)	0.3
  (1, 1)	0.2
  (0, 2)	0.4
  (1, 2)	0.5
  (2, 2)	0.3
  (0, 3)	0.05
  (1, 3)	0.15
  (2, 3)	0.1
(3, 1)
c: [[-0.45]
 [-0.53]
 [-0.38]]
length of c: 3
n_funds: 3
A: [[1. 1. 1.]]


ValueError: Incorrect dimension of q

In [9]:
import osqp
osqp.__version__

'0.6.7.post3'

In [20]:
np.array([1, 1]).shape

(2,)

In [55]:
P = sparse.csc_matrix([[4, 1], [1, 2]])
q = np.array([1, 1])
A = sparse.csc_matrix([[1, 1], [1, 0], [0, 1]])
l = np.array([1, 0, 0])
u = np.array([1, 0.7, 0.7])

# Create an OSQP object
#prob = osqp.OSQP()

# Setup workspace and change alpha parameter
#prob.setup(P, q, A, l, u, alpha=1.0)

# Solve problem
#res = prob.solve()

# Define problem settings
# Taken from: https://github.com/osqp/miosqp/blob/master/examples/random_miqp/run_example.py#L98
miosqp_settings = {
    # integer feasibility tolerance
#    'eps_int_feas': 1e-03,
    # maximum number of iterations
#    'max_iter_bb': 1000,
    'max_iter': 1000,
    # tree exploration rule
    #   [0] depth first
    #   [1] two-phase: depth first until first incumbent and then  best bound
#    'tree_explor_rule': 1,
    # branching rule
    #   [0] max fractional part
#    'branching_rule': 0,
    'verbose': True,
#    'print_interval': 1
}

osqp_settings = {
    'eps_abs': 1e-03,
    'eps_rel': 1e-03,
    'eps_prim_inf': 1e-04,
    'verbose': True}

miosqp_solver = miosqp.MIOSQP()
miosqp_solver.setup(P, q, A, l, u,
                    i_idx = [], i_l = [], i_u = [], 
                    settings = osqp_settings,
                    qp_settings = miosqp_settings)  # No integer constraints for now
result = miosqp_solver.solve()


-----------------------------------------------------------------
           OSQP v0.6.3  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2021
-----------------------------------------------------------------
problem:  variables n = 2, constraints m = 3
          nnz(P) + nnz(A) = 7
settings: linear system solver = qdldl,
          eps_abs = 1.0e-03, eps_rel = 1.0e-03,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 1000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: off, time_limit: off

     Nodes      |           Current Node        |             Objective Bounds             |   Cur Node
Explr	Unexplr	|      Obj	Depth	IntInf  |    Lower	   Upper	    Gap    |     Iter


KeyError: 'max_iter_bb'

In [56]:
import osqp
import numpy as np
from scipy import sparse

# Define problem data
P = sparse.csc_matrix([[4, 1], [1, 2]])
q = np.array([1, 1])
A = sparse.csc_matrix([[1, 1], [1, 0], [0, 1]])
l = np.array([1, 0, 0])
u = np.array([1, 0.7, 0.7])

# Create an OSQP object
prob = osqp.OSQP()

# Setup workspace and change alpha parameter
prob.setup(P, q, A, l, u, alpha=1.0)

# Solve problem
res = prob.solve()

-----------------------------------------------------------------
           OSQP v0.6.3  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2021
-----------------------------------------------------------------
problem:  variables n = 2, constraints m = 3
          nnz(P) + nnz(A) = 7
settings: linear system solver = qdldl,
          eps_abs = 1.0e-03, eps_rel = 1.0e-03,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.00, max_iter = 4000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: off, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -4.9384e-03   1.00e+00   2.00e+02   1.00e-01   7.64e-05s
  50   1.8800e+00   1.91e-07   7.50e-07   1.38e+00   2.18e-04s

status:               solved
number of iterations:

In [63]:
# From: https://github.com/osqp/miosqp/blob/master/examples/random_miqp/run_example.py

import scipy as sp
import scipy.sparse as spa
import numpy as np

# Get dimensions
n = 10 # n_vec[i]
m = 5  # m_vec[i]
p = 2  # p_vec[i]
dns_level = 0.7         # density level for sparse matrices

# Generate random Matrices
Pt = spa.random(n, n, density=dns_level)
P = spa.csc_matrix(np.dot(Pt, Pt.T))
q = spa.random(n)
A = spa.random(m, n, density=dns_level)
u = 2 + sp.rand(m)
l = -2 + sp.rand(m)

# Enforce [0, 1] bounds on variables
i_l = np.zeros(p)
i_u = np.ones(p)
#  A, l, u = miosqp.add_bounds(i_idx, 0., 1., A, l, u)


TypeError: random() missing 1 required positional argument: 'n'